In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import cross_val_score

In [2]:
# charts options
plt.figure(figsize=(15,5));
sns.set(rc={'figure.figsize':(15,5), 'xtick.direction':'in'});

# pandas options
pd.set_option('display.max_columns', 999)

<Figure size 1080x360 with 0 Axes>

In [49]:
df = pd.read_json('base-dados-clube-da-aposta.json')

ordens_coluna = ['home_name', 'away_name', 'home_score', 'away_score', 'final_result',
       'time', 'home_pos', 'away_pos', 'round', 'home_last5all_home',
       'home_last5all_home_win', 'home_last5all_home_draw',
       'home_last5all_home_lose', 'away_last5all_away',
       'away_last5all_away_win', 'away_last5all_away_draw',
       'away_last5all_away_lose', 'last5all_home_away_dif', 'fifa_home_ova',
       'fifa_home_att', 'fifa_home_mid', 'fifa_home_def', 'fifa_away_ova',
       'fifa_away_att', 'fifa_away_mid', 'fifa_away_def', 'elo_home_score',
       'elo_away_score', 'tfm_value_home', 'tfm_value_away']
df = df[ordens_coluna]

df = df.sort_index()

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5163 entries, 0 to 5162
Data columns (total 30 columns):
home_name                  5163 non-null object
away_name                  5163 non-null object
home_score                 5163 non-null int64
away_score                 5163 non-null int64
final_result               5163 non-null object
time                       5163 non-null int64
home_pos                   5163 non-null int64
away_pos                   5163 non-null int64
round                      5163 non-null int64
home_last5all_home         5163 non-null int64
home_last5all_home_win     5163 non-null int64
home_last5all_home_draw    5163 non-null int64
home_last5all_home_lose    5163 non-null int64
away_last5all_away         5163 non-null int64
away_last5all_away_win     5163 non-null int64
away_last5all_away_draw    5163 non-null int64
away_last5all_away_lose    5163 non-null int64
last5all_home_away_dif     5163 non-null int64
fifa_home_ova              5163 non-null int

# Discussão sobre features

Algumas colunas podem ser consideradas como componentes para formar uma Time serie. Julguei `round`, `time` e `home/away_score`. 

As 2 primeiras ok, mas porque a última? O mesmo time fabrica um histórico durante o campeonato e durante os anos isso pode se tornar uma informação muito importante. É comum noticiar que o time x não ganha do time y há tanto tempo, principalmente em clássicos.


## Pré-processamento

### 1) Convertendo input de dados categóricos

Alguns algoritmos trabalham melhor com números

In [51]:
# # convertendo resultado do jogo para numero
# def transforma_final_result(x):
#     if x=='H':
#         return 0
#     if x=='A':
#         return 1
#     if x=='D':
#         return 2

In [52]:
# df['final_result_trans'] = df.final_result.apply(lambda x: transforma_final_result(x))
# 
# df.drop('final_result', axis=1, inplace=True)

In [53]:
# convertendo o nome dos times para numeros

"""
Nesse caso vamos usar o LabelEncoder do sklearn pois existem centenas de times.
Essa transformação é necessária para que possamos fazer isso com novos dados.
"""

dft = df[['home_name', 'away_name']]
le = preprocessing.LabelEncoder()

name_trans = dft.apply(le.fit_transform)

# para fazer a conversao inversa. Agora nao precisarei, mas depois sim.
# name_trans.apply(le.inverse_transform)

# substituindo os valores
df.home_name = name_trans.home_name
df.away_name = name_trans.away_name

### 2) Removendo tempo e rodada

O score Elo leva em consideração o último jogo do time. Dependendo do resultado esse valor é alterado. Então
desconsiderei o tempo pois ele já está implicito no score Elo.

Esse conjunto de dados é uma Time series, porém o modelo poderia ficar muito complexo devido à variedade de ligas e os dias
que acontecem os jogos.

Sabemos que uma rodada ao final do campeonato com o time já campeão pode significar uma derrota para um time fraco, porém
esse é um comportamento anormal e prever isso poderia também tornar o modelo complexo. Para esses casos o feeling do apostador
vale mais do que qualquer previsão.


In [54]:
# removendo a coluna tempo e rodada

df.drop(['time', 'round'], axis=1, inplace=True)

### 3) last5all: Juntando 3 colunas em uma

Existem 3 colunas com essa característica: win, draw e lose. Juntando essas 3 colunas em uma calculando o aproveitamento de pontos.

In [55]:
def aprov_last5_home(col):
    aprov_win = col['home_last5all_home_win']*3
    aprov_draw = col['home_last5all_home_draw']*1
    
    return int((aprov_win+aprov_draw)/15*100)

def aprov_last5_away(col):
    aprov_win = col['away_last5all_away_win']*3
    aprov_draw = col['away_last5all_away_draw']*1
    
    return int((aprov_win+aprov_draw)/15*100)

In [56]:
df['aprov_5_home'] = df[['home_last5all_home_win', 'home_last5all_home_draw']].apply(lambda x: aprov_last5_home(x), axis=1)
df['aprov_5_away'] = df[['away_last5all_away_win', 'away_last5all_away_draw']].apply(lambda x: aprov_last5_away(x), axis=1)

In [57]:
df.drop(['home_last5all_home_win', 'home_last5all_home_draw', 'home_last5all_home_lose',
         'away_last5all_away_win', 'away_last5all_away_draw', 'away_last5all_away_lose'], axis=1, inplace=True)

Completado o pré-processamento o dataset ficou assim:

In [58]:
df.head(8)

,home_name,away_name,home_score,away_score,final_result,home_pos,away_pos,home_last5all_home,away_last5all_away,last5all_home_away_dif,fifa_home_ova,fifa_home_att,fifa_home_mid,fifa_home_def,fifa_away_ova,fifa_away_att,fifa_away_mid,fifa_away_def,elo_home_score,elo_away_score,tfm_value_home,tfm_value_away,aprov_5_home,aprov_5_away
0,87,157,3,1,H,1,11,9,-8,17,80,79,81,80,70,70,70,69,1813.906738,1392.986084,277980000.0,9450000.0,100,26
1,219,78,2,1,H,18,15,2,-7,9,70,70,71,69,67,69,67,67,1513.179932,1475.512939,10900000.0,9600000.0,60,26
2,163,45,1,1,D,7,2,1,3,-2,65,70,65,65,68,68,68,69,1384.888916,1495.836548,6700000.0,15850000.0,60,53
3,137,148,3,1,H,2,6,11,-1,12,82,84,83,81,83,84,82,79,1930.906372,1841.372681,913000000.0,835000000.0,100,46
4,207,18,3,2,H,19,5,-3,5,-8,77,76,77,77,82,85,83,80,1606.425171,1856.242188,266100000.0,574000000.0,26,73
5,45,61,1,2,A,13,4,3,2,1,76,77,75,76,83,84,84,83,1636.926270,1858.815918,177000000.0,892250000.0,66,46
6,97,243,0,2,A,20,11,-6,4,-10,76,76,76,73,78,79,77,77,1573.946045,1689.784790,250500000.0,287500000.0,33,53
7,116,161,0,1,A,18,15,-3,0,-3,75,74,74,75,76,76,76,76,1546.040771,1625.302979,120600000.0,174000000.0,26,40


## 2) Separando o dataset

In [59]:
y = df['final_result']
x = df.drop('final_result', axis=1)

## Fazendo o benchmarking

Vamos aplicar um algoritmo da forma que os dados estão para fazermos uma comparação depois de realizar mais algumas transformações.

Vamos usar o RandomForest para verificar a acertividade inicial do modelo, bem como as features importances.

In [154]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=29384)

In [155]:
clf = RandomForestClassifier(n_estimators=100, n_jobs=-1, max_depth=8, min_samples_leaf=50, random_state=99, verbose=0)
clf.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=8, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=50, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=99, verbose=0, warm_start=False)

In [156]:
result = clf.predict(x_test)

In [157]:
accuracy_score(y_test, result)

0.9110251450676983

In [158]:
# Função que monta um dataframe para exibir as feature importances
def view_feature_importances(feat, impor):
    return pd.DataFrame({'features': feat, 'importances': impor}).sort_values(by='importances', ascending=False).set_index('features')

## Tests:

In [159]:
x_test

,home_name,away_name,home_score,away_score,home_pos,away_pos,home_last5all_home,away_last5all_away,last5all_home_away_dif,fifa_home_ova,fifa_home_att,fifa_home_mid,fifa_home_def,fifa_away_ova,fifa_away_att,fifa_away_mid,fifa_away_def,elo_home_score,elo_away_score,tfm_value_home,tfm_value_away,aprov_5_home,aprov_5_away
2140,210,94,0,0,10,18,3,-4,7,67,65,67,68,69,70,68,68,1430.848145,1437.770264,14130000.0,14900000.0,80,26
769,59,25,1,2,6,12,4,0,4,66,66,67,64,68,70,69,65,1430.055664,1392.379761,10080000.0,16000000.0,80,40
3569,219,166,1,1,17,20,2,-4,6,69,71,70,68,69,69,70,69,1532.363281,1545.082153,8500000.0,9300000.0,60,20
2258,174,151,2,1,8,1,5,7,-2,66,65,66,66,70,70,69,71,1440.423828,1518.113892,8200000.0,22550000.0,80,100
86,59,25,0,0,12,13,1,2,-1,64,66,63,63,69,69,70,67,1380.316284,1366.218384,9180000.0,18900000.0,46,46
4031,69,133,2,2,16,11,-2,-4,2,77,80,77,77,78,78,78,76,1639.270020,1721.494995,186500000.0,205300000.0,60,20
1115,40,81,2,2,6,7,2,-7,9,76,76,75,75,79,77,80,79,1684.709717,1699.965332,164250000.0,400000000.0,60,13
38,95,197,2,0,18,12,-4,-1,-3,72,72,72,71,74,75,73,74,1496.192505,1630.267822,41950000.0,88150000.0,20,26
4114,26,23,1,0,1,2,9,6,3,85,89,85,84,83,85,82,83,2032.626587,1945.089478,957000000.0,587000000.0,86,80
3005,48,98,0,5,13,4,1,-6,7,72,72,72,71,77,78,77,74,1436.719238,1560.959717,44300000.0,100100000.0,46,26


In [160]:
import itertools

a = [0, 1, 2, 3, 4]
b = [0, 1, 2, 3, 4]

comb_results = list(itertools.product(a, b))

#for p in itertools.product(a, b):
#    print(p)
#(0, 0),
#(0, 1),
#(0, 2),
#(0, 3),
#(0, 4),
#(1, 0),

In [161]:
x_test.shape

(517, 23)

In [219]:
final = []
def add_result(row, results):
    for home, away in results:
        row_copy = row.copy()
        row_copy['home_score'] = home
        row_copy['away_score'] = away
        final.append(row_copy)
        print(row['home_score'], row['away_score'], home, away)
    print('acabou linha')
    #return row

In [220]:
x_test

,home_name,away_name,home_score,away_score,home_pos,away_pos,home_last5all_home,away_last5all_away,last5all_home_away_dif,fifa_home_ova,fifa_home_att,fifa_home_mid,fifa_home_def,fifa_away_ova,fifa_away_att,fifa_away_mid,fifa_away_def,elo_home_score,elo_away_score,tfm_value_home,tfm_value_away,aprov_5_home,aprov_5_away
2140,210.0,94.0,0.0,0.0,10.0,18.0,3.0,-4.0,7.0,67.0,65.0,67.0,68.0,69.0,70.0,68.0,68.0,1430.848145,1437.770264,14130000.0,14900000.0,80.0,26.0
769,59.0,25.0,1.0,2.0,6.0,12.0,4.0,0.0,4.0,66.0,66.0,67.0,64.0,68.0,70.0,69.0,65.0,1430.055664,1392.379761,10080000.0,16000000.0,80.0,40.0
3569,219.0,166.0,1.0,1.0,17.0,20.0,2.0,-4.0,6.0,69.0,71.0,70.0,68.0,69.0,69.0,70.0,69.0,1532.363281,1545.082153,8500000.0,9300000.0,60.0,20.0
2258,174.0,151.0,2.0,1.0,8.0,1.0,5.0,7.0,-2.0,66.0,65.0,66.0,66.0,70.0,70.0,69.0,71.0,1440.423828,1518.113892,8200000.0,22550000.0,80.0,100.0
86,59.0,25.0,0.0,0.0,12.0,13.0,1.0,2.0,-1.0,64.0,66.0,63.0,63.0,69.0,69.0,70.0,67.0,1380.316284,1366.218384,9180000.0,18900000.0,46.0,46.0
4031,69.0,133.0,2.0,2.0,16.0,11.0,-2.0,-4.0,2.0,77.0,80.0,77.0,77.0,78.0,78.0,78.0,76.0,1639.270020,1721.494995,186500000.0,205300000.0,60.0,20.0
1115,40.0,81.0,2.0,2.0,6.0,7.0,2.0,-7.0,9.0,76.0,76.0,75.0,75.0,79.0,77.0,80.0,79.0,1684.709717,1699.965332,164250000.0,400000000.0,60.0,13.0
38,95.0,197.0,2.0,0.0,18.0,12.0,-4.0,-1.0,-3.0,72.0,72.0,72.0,71.0,74.0,75.0,73.0,74.0,1496.192505,1630.267822,41950000.0,88150000.0,20.0,26.0
4114,26.0,23.0,1.0,0.0,1.0,2.0,9.0,6.0,3.0,85.0,89.0,85.0,84.0,83.0,85.0,82.0,83.0,2032.626587,1945.089478,957000000.0,587000000.0,86.0,80.0
3005,48.0,98.0,0.0,5.0,13.0,4.0,1.0,-6.0,7.0,72.0,72.0,72.0,71.0,77.0,78.0,77.0,74.0,1436.719238,1560.959717,44300000.0,100100000.0,46.0,26.0


In [221]:
final = []
x_test = x_test.apply(lambda x: add_result(x, comb_results), axis=1)

0.0 0.0 0 0
0.0 0.0 0 1
0.0 0.0 0 2
0.0 0.0 0 3
0.0 0.0 0 4
0.0 0.0 1 0
0.0 0.0 1 1
0.0 0.0 1 2
0.0 0.0 1 3
0.0 0.0 1 4
0.0 0.0 2 0
0.0 0.0 2 1
0.0 0.0 2 2
0.0 0.0 2 3
0.0 0.0 2 4
0.0 0.0 3 0
0.0 0.0 3 1
0.0 0.0 3 2
0.0 0.0 3 3
0.0 0.0 3 4
0.0 0.0 4 0
0.0 0.0 4 1
0.0 0.0 4 2
0.0 0.0 4 3
0.0 0.0 4 4
acabou linha
1.0 2.0 0 0
1.0 2.0 0 1
1.0 2.0 0 2
1.0 2.0 0 3
1.0 2.0 0 4
1.0 2.0 1 0
1.0 2.0 1 1
1.0 2.0 1 2
1.0 2.0 1 3
1.0 2.0 1 4
1.0 2.0 2 0
1.0 2.0 2 1
1.0 2.0 2 2
1.0 2.0 2 3
1.0 2.0 2 4
1.0 2.0 3 0
1.0 2.0 3 1
1.0 2.0 3 2
1.0 2.0 3 3
1.0 2.0 3 4
1.0 2.0 4 0
1.0 2.0 4 1
1.0 2.0 4 2
1.0 2.0 4 3
1.0 2.0 4 4
acabou linha
1.0 1.0 0 0
1.0 1.0 0 1
1.0 1.0 0 2
1.0 1.0 0 3
1.0 1.0 0 4
1.0 1.0 1 0
1.0 1.0 1 1
1.0 1.0 1 2
1.0 1.0 1 3
1.0 1.0 1 4
1.0 1.0 2 0
1.0 1.0 2 1
1.0 1.0 2 2
1.0 1.0 2 3
1.0 1.0 2 4
1.0 1.0 3 0
1.0 1.0 3 1
1.0 1.0 3 2
1.0 1.0 3 3
1.0 1.0 3 4
1.0 1.0 4 0
1.0 1.0 4 1
1.0 1.0 4 2
1.0 1.0 4 3
1.0 1.0 4 4
acabou linha
2.0 1.0 0 0
2.0 1.0 0 1
2.0 1.0 0 2
2.0 1.0 0 3
2.0 1.0 0 4
2

0.0 2.0 1 3
0.0 2.0 1 4
0.0 2.0 2 0
0.0 2.0 2 1
0.0 2.0 2 2
0.0 2.0 2 3
0.0 2.0 2 4
0.0 2.0 3 0
0.0 2.0 3 1
0.0 2.0 3 2
0.0 2.0 3 3
0.0 2.0 3 4
0.0 2.0 4 0
0.0 2.0 4 1
0.0 2.0 4 2
0.0 2.0 4 3
0.0 2.0 4 4
acabou linha
0.0 1.0 0 0
0.0 1.0 0 1
0.0 1.0 0 2
0.0 1.0 0 3
0.0 1.0 0 4
0.0 1.0 1 0
0.0 1.0 1 1
0.0 1.0 1 2
0.0 1.0 1 3
0.0 1.0 1 4
0.0 1.0 2 0
0.0 1.0 2 1
0.0 1.0 2 2
0.0 1.0 2 3
0.0 1.0 2 4
0.0 1.0 3 0
0.0 1.0 3 1
0.0 1.0 3 2
0.0 1.0 3 3
0.0 1.0 3 4
0.0 1.0 4 0
0.0 1.0 4 1
0.0 1.0 4 2
0.0 1.0 4 3
0.0 1.0 4 4
acabou linha
1.0 1.0 0 0
1.0 1.0 0 1
1.0 1.0 0 2
1.0 1.0 0 3
1.0 1.0 0 4
1.0 1.0 1 0
1.0 1.0 1 1
1.0 1.0 1 2
1.0 1.0 1 3
1.0 1.0 1 4
1.0 1.0 2 0
1.0 1.0 2 1
1.0 1.0 2 2
1.0 1.0 2 3
1.0 1.0 2 4
1.0 1.0 3 0
1.0 1.0 3 1
1.0 1.0 3 2
1.0 1.0 3 3
1.0 1.0 3 4
1.0 1.0 4 0
1.0 1.0 4 1
1.0 1.0 4 2
1.0 1.0 4 3
1.0 1.0 4 4
acabou linha
2.0 1.0 0 0
2.0 1.0 0 1
2.0 1.0 0 2
2.0 1.0 0 3
2.0 1.0 0 4
2.0 1.0 1 0
2.0 1.0 1 1
2.0 1.0 1 2
2.0 1.0 1 3
2.0 1.0 1 4
2.0 1.0 2 0
2.0 1.0 2 1
2.0 1.0 2 2
2

1.0 1.0 3 4
1.0 1.0 4 0
1.0 1.0 4 1
1.0 1.0 4 2
1.0 1.0 4 3
1.0 1.0 4 4
acabou linha
4.0 1.0 0 0
4.0 1.0 0 1
4.0 1.0 0 2
4.0 1.0 0 3
4.0 1.0 0 4
4.0 1.0 1 0
4.0 1.0 1 1
4.0 1.0 1 2
4.0 1.0 1 3
4.0 1.0 1 4
4.0 1.0 2 0
4.0 1.0 2 1
4.0 1.0 2 2
4.0 1.0 2 3
4.0 1.0 2 4
4.0 1.0 3 0
4.0 1.0 3 1
4.0 1.0 3 2
4.0 1.0 3 3
4.0 1.0 3 4
4.0 1.0 4 0
4.0 1.0 4 1
4.0 1.0 4 2
4.0 1.0 4 3
4.0 1.0 4 4
acabou linha
2.0 3.0 0 0
2.0 3.0 0 1
2.0 3.0 0 2
2.0 3.0 0 3
2.0 3.0 0 4
2.0 3.0 1 0
2.0 3.0 1 1
2.0 3.0 1 2
2.0 3.0 1 3
2.0 3.0 1 4
2.0 3.0 2 0
2.0 3.0 2 1
2.0 3.0 2 2
2.0 3.0 2 3
2.0 3.0 2 4
2.0 3.0 3 0
2.0 3.0 3 1
2.0 3.0 3 2
2.0 3.0 3 3
2.0 3.0 3 4
2.0 3.0 4 0
2.0 3.0 4 1
2.0 3.0 4 2
2.0 3.0 4 3
2.0 3.0 4 4
acabou linha
1.0 0.0 0 0
1.0 0.0 0 1
1.0 0.0 0 2
1.0 0.0 0 3
1.0 0.0 0 4
1.0 0.0 1 0
1.0 0.0 1 1
1.0 0.0 1 2
1.0 0.0 1 3
1.0 0.0 1 4
1.0 0.0 2 0
1.0 0.0 2 1
1.0 0.0 2 2
1.0 0.0 2 3
1.0 0.0 2 4
1.0 0.0 3 0
1.0 0.0 3 1
1.0 0.0 3 2
1.0 0.0 3 3
1.0 0.0 3 4
1.0 0.0 4 0
1.0 0.0 4 1
1.0 0.0 4 2
1.0 0.0 4 3
1

0.0 0.0 1 4
0.0 0.0 2 0
0.0 0.0 2 1
0.0 0.0 2 2
0.0 0.0 2 3
0.0 0.0 2 4
0.0 0.0 3 0
0.0 0.0 3 1
0.0 0.0 3 2
0.0 0.0 3 3
0.0 0.0 3 4
0.0 0.0 4 0
0.0 0.0 4 1
0.0 0.0 4 2
0.0 0.0 4 3
0.0 0.0 4 4
acabou linha
5.0 0.0 0 0
5.0 0.0 0 1
5.0 0.0 0 2
5.0 0.0 0 3
5.0 0.0 0 4
5.0 0.0 1 0
5.0 0.0 1 1
5.0 0.0 1 2
5.0 0.0 1 3
5.0 0.0 1 4
5.0 0.0 2 0
5.0 0.0 2 1
5.0 0.0 2 2
5.0 0.0 2 3
5.0 0.0 2 4
5.0 0.0 3 0
5.0 0.0 3 1
5.0 0.0 3 2
5.0 0.0 3 3
5.0 0.0 3 4
5.0 0.0 4 0
5.0 0.0 4 1
5.0 0.0 4 2
5.0 0.0 4 3
5.0 0.0 4 4
acabou linha
1.0 1.0 0 0
1.0 1.0 0 1
1.0 1.0 0 2
1.0 1.0 0 3
1.0 1.0 0 4
1.0 1.0 1 0
1.0 1.0 1 1
1.0 1.0 1 2
1.0 1.0 1 3
1.0 1.0 1 4
1.0 1.0 2 0
1.0 1.0 2 1
1.0 1.0 2 2
1.0 1.0 2 3
1.0 1.0 2 4
1.0 1.0 3 0
1.0 1.0 3 1
1.0 1.0 3 2
1.0 1.0 3 3
1.0 1.0 3 4
1.0 1.0 4 0
1.0 1.0 4 1
1.0 1.0 4 2
1.0 1.0 4 3
1.0 1.0 4 4
acabou linha
1.0 0.0 0 0
1.0 0.0 0 1
1.0 0.0 0 2
1.0 0.0 0 3
1.0 0.0 0 4
1.0 0.0 1 0
1.0 0.0 1 1
1.0 0.0 1 2
1.0 0.0 1 3
1.0 0.0 1 4
1.0 0.0 2 0
1.0 0.0 2 1
1.0 0.0 2 2
1.0 0.0 2 3
1

1.0 3.0 0 0
1.0 3.0 0 1
1.0 3.0 0 2
1.0 3.0 0 3
1.0 3.0 0 4
1.0 3.0 1 0
1.0 3.0 1 1
1.0 3.0 1 2
1.0 3.0 1 3
1.0 3.0 1 4
1.0 3.0 2 0
1.0 3.0 2 1
1.0 3.0 2 2
1.0 3.0 2 3
1.0 3.0 2 4
1.0 3.0 3 0
1.0 3.0 3 1
1.0 3.0 3 2
1.0 3.0 3 3
1.0 3.0 3 4
1.0 3.0 4 0
1.0 3.0 4 1
1.0 3.0 4 2
1.0 3.0 4 3
1.0 3.0 4 4
acabou linha
3.0 1.0 0 0
3.0 1.0 0 1
3.0 1.0 0 2
3.0 1.0 0 3
3.0 1.0 0 4
3.0 1.0 1 0
3.0 1.0 1 1
3.0 1.0 1 2
3.0 1.0 1 3
3.0 1.0 1 4
3.0 1.0 2 0
3.0 1.0 2 1
3.0 1.0 2 2
3.0 1.0 2 3
3.0 1.0 2 4
3.0 1.0 3 0
3.0 1.0 3 1
3.0 1.0 3 2
3.0 1.0 3 3
3.0 1.0 3 4
3.0 1.0 4 0
3.0 1.0 4 1
3.0 1.0 4 2
3.0 1.0 4 3
3.0 1.0 4 4
acabou linha
3.0 0.0 0 0
3.0 0.0 0 1
3.0 0.0 0 2
3.0 0.0 0 3
3.0 0.0 0 4
3.0 0.0 1 0
3.0 0.0 1 1
3.0 0.0 1 2
3.0 0.0 1 3
3.0 0.0 1 4
3.0 0.0 2 0
3.0 0.0 2 1
3.0 0.0 2 2
3.0 0.0 2 3
3.0 0.0 2 4
3.0 0.0 3 0
3.0 0.0 3 1
3.0 0.0 3 2
3.0 0.0 3 3
3.0 0.0 3 4
3.0 0.0 4 0
3.0 0.0 4 1
3.0 0.0 4 2
3.0 0.0 4 3
3.0 0.0 4 4
acabou linha
1.0 1.0 0 0
1.0 1.0 0 1
1.0 1.0 0 2
1.0 1.0 0 3
1.0 1.0 0 4
1

...

2.0 2.0 2 1
2.0 2.0 2 2
2.0 2.0 2 3
2.0 2.0 2 4
2.0 2.0 3 0
2.0 2.0 3 1
2.0 2.0 3 2
2.0 2.0 3 3
2.0 2.0 3 4
2.0 2.0 4 0
2.0 2.0 4 1
2.0 2.0 4 2
2.0 2.0 4 3
2.0 2.0 4 4
acabou linha
2.0 1.0 0 0
2.0 1.0 0 1
2.0 1.0 0 2
2.0 1.0 0 3
2.0 1.0 0 4
2.0 1.0 1 0
2.0 1.0 1 1
2.0 1.0 1 2
2.0 1.0 1 3
2.0 1.0 1 4
2.0 1.0 2 0
2.0 1.0 2 1
2.0 1.0 2 2
2.0 1.0 2 3
2.0 1.0 2 4
2.0 1.0 3 0
2.0 1.0 3 1
2.0 1.0 3 2
2.0 1.0 3 3
2.0 1.0 3 4
2.0 1.0 4 0
2.0 1.0 4 1
2.0 1.0 4 2
2.0 1.0 4 3
2.0 1.0 4 4
acabou linha
1.0 1.0 0 0
1.0 1.0 0 1
1.0 1.0 0 2
1.0 1.0 0 3
1.0 1.0 0 4
1.0 1.0 1 0
1.0 1.0 1 1
1.0 1.0 1 2
1.0 1.0 1 3
1.0 1.0 1 4
1.0 1.0 2 0
1.0 1.0 2 1
1.0 1.0 2 2
1.0 1.0 2 3
1.0 1.0 2 4
1.0 1.0 3 0
1.0 1.0 3 1
1.0 1.0 3 2
1.0 1.0 3 3
1.0 1.0 3 4
1.0 1.0 4 0
1.0 1.0 4 1
1.0 1.0 4 2
1.0 1.0 4 3
1.0 1.0 4 4
acabou linha
1.0 1.0 0 0
1.0 1.0 0 1
1.0 1.0 0 2
1.0 1.0 0 3
1.0 1.0 0 4
1.0 1.0 1 0
1.0 1.0 1 1
1.0 1.0 1 2
1.0 1.0 1 3
1.0 1.0 1 4
1.0 1.0 2 0
1.0 1.0 2 1
1.0 1.0 2 2
1.0 1.0 2 3
1.0 1.0 2 4
1.0 1.0 3 0
1

2.0 1.0 0 4
2.0 1.0 1 0
2.0 1.0 1 1
2.0 1.0 1 2
2.0 1.0 1 3
2.0 1.0 1 4
2.0 1.0 2 0
2.0 1.0 2 1
2.0 1.0 2 2
2.0 1.0 2 3
2.0 1.0 2 4
2.0 1.0 3 0
2.0 1.0 3 1
2.0 1.0 3 2
2.0 1.0 3 3
2.0 1.0 3 4
2.0 1.0 4 0
2.0 1.0 4 1
2.0 1.0 4 2
2.0 1.0 4 3
2.0 1.0 4 4
acabou linha
1.0 4.0 0 0
1.0 4.0 0 1
1.0 4.0 0 2
1.0 4.0 0 3
1.0 4.0 0 4
1.0 4.0 1 0
1.0 4.0 1 1
1.0 4.0 1 2
1.0 4.0 1 3
1.0 4.0 1 4
1.0 4.0 2 0
1.0 4.0 2 1
1.0 4.0 2 2
1.0 4.0 2 3
1.0 4.0 2 4
1.0 4.0 3 0
1.0 4.0 3 1
1.0 4.0 3 2
1.0 4.0 3 3
1.0 4.0 3 4
1.0 4.0 4 0
1.0 4.0 4 1
1.0 4.0 4 2
1.0 4.0 4 3
1.0 4.0 4 4
acabou linha
3.0 1.0 0 0
3.0 1.0 0 1
3.0 1.0 0 2
3.0 1.0 0 3
3.0 1.0 0 4
3.0 1.0 1 0
3.0 1.0 1 1
3.0 1.0 1 2
3.0 1.0 1 3
3.0 1.0 1 4
3.0 1.0 2 0
3.0 1.0 2 1
3.0 1.0 2 2
3.0 1.0 2 3
3.0 1.0 2 4
3.0 1.0 3 0
3.0 1.0 3 1
3.0 1.0 3 2
3.0 1.0 3 3
3.0 1.0 3 4
3.0 1.0 4 0
3.0 1.0 4 1
3.0 1.0 4 2
3.0 1.0 4 3
3.0 1.0 4 4
acabou linha
2.0 0.0 0 0
2.0 0.0 0 1
2.0 0.0 0 2
2.0 0.0 0 3
2.0 0.0 0 4
2.0 0.0 1 0
2.0 0.0 1 1
2.0 0.0 1 2
2.0 0.0 1 3
2

3.0 0.0 4 1
3.0 0.0 4 2
3.0 0.0 4 3
3.0 0.0 4 4
acabou linha
1.0 0.0 0 0
1.0 0.0 0 1
1.0 0.0 0 2
1.0 0.0 0 3
1.0 0.0 0 4
1.0 0.0 1 0
1.0 0.0 1 1
1.0 0.0 1 2
1.0 0.0 1 3
1.0 0.0 1 4
1.0 0.0 2 0
1.0 0.0 2 1
1.0 0.0 2 2
1.0 0.0 2 3
1.0 0.0 2 4
1.0 0.0 3 0
1.0 0.0 3 1
1.0 0.0 3 2
1.0 0.0 3 3
1.0 0.0 3 4
1.0 0.0 4 0
1.0 0.0 4 1
1.0 0.0 4 2
1.0 0.0 4 3
1.0 0.0 4 4
acabou linha
2.0 2.0 0 0
2.0 2.0 0 1
2.0 2.0 0 2
2.0 2.0 0 3
2.0 2.0 0 4
2.0 2.0 1 0
2.0 2.0 1 1
2.0 2.0 1 2
2.0 2.0 1 3
2.0 2.0 1 4
2.0 2.0 2 0
2.0 2.0 2 1
2.0 2.0 2 2
2.0 2.0 2 3
2.0 2.0 2 4
2.0 2.0 3 0
2.0 2.0 3 1
2.0 2.0 3 2
2.0 2.0 3 3
2.0 2.0 3 4
2.0 2.0 4 0
2.0 2.0 4 1
2.0 2.0 4 2
2.0 2.0 4 3
2.0 2.0 4 4
acabou linha
1.0 1.0 0 0
1.0 1.0 0 1
1.0 1.0 0 2
1.0 1.0 0 3
1.0 1.0 0 4
1.0 1.0 1 0
1.0 1.0 1 1
1.0 1.0 1 2
1.0 1.0 1 3
1.0 1.0 1 4
1.0 1.0 2 0
1.0 1.0 2 1
1.0 1.0 2 2
1.0 1.0 2 3
1.0 1.0 2 4
1.0 1.0 3 0
1.0 1.0 3 1
1.0 1.0 3 2
1.0 1.0 3 3
1.0 1.0 3 4
1.0 1.0 4 0
1.0 1.0 4 1
1.0 1.0 4 2
1.0 1.0 4 3
1.0 1.0 4 4
acabou linha


2.0 1.0 2 3
2.0 1.0 2 4
2.0 1.0 3 0
2.0 1.0 3 1
2.0 1.0 3 2
2.0 1.0 3 3
2.0 1.0 3 4
2.0 1.0 4 0
2.0 1.0 4 1
2.0 1.0 4 2
2.0 1.0 4 3
2.0 1.0 4 4
acabou linha
0.0 1.0 0 0
0.0 1.0 0 1
0.0 1.0 0 2
0.0 1.0 0 3
0.0 1.0 0 4
0.0 1.0 1 0
0.0 1.0 1 1
0.0 1.0 1 2
0.0 1.0 1 3
0.0 1.0 1 4
0.0 1.0 2 0
0.0 1.0 2 1
0.0 1.0 2 2
0.0 1.0 2 3
0.0 1.0 2 4
0.0 1.0 3 0
0.0 1.0 3 1
0.0 1.0 3 2
0.0 1.0 3 3
0.0 1.0 3 4
0.0 1.0 4 0
0.0 1.0 4 1
0.0 1.0 4 2
0.0 1.0 4 3
0.0 1.0 4 4
acabou linha
1.0 2.0 0 0
1.0 2.0 0 1
1.0 2.0 0 2
1.0 2.0 0 3
1.0 2.0 0 4
1.0 2.0 1 0
1.0 2.0 1 1
1.0 2.0 1 2
1.0 2.0 1 3
1.0 2.0 1 4
1.0 2.0 2 0
1.0 2.0 2 1
1.0 2.0 2 2
1.0 2.0 2 3
1.0 2.0 2 4
1.0 2.0 3 0
1.0 2.0 3 1
1.0 2.0 3 2
1.0 2.0 3 3
1.0 2.0 3 4
1.0 2.0 4 0
1.0 2.0 4 1
1.0 2.0 4 2
1.0 2.0 4 3
1.0 2.0 4 4
acabou linha
0.0 2.0 0 0
0.0 2.0 0 1
0.0 2.0 0 2
0.0 2.0 0 3
0.0 2.0 0 4
0.0 2.0 1 0
0.0 2.0 1 1
0.0 2.0 1 2
0.0 2.0 1 3
0.0 2.0 1 4
0.0 2.0 2 0
0.0 2.0 2 1
0.0 2.0 2 2
0.0 2.0 2 3
0.0 2.0 2 4
0.0 2.0 3 0
0.0 2.0 3 1
0.0 2.0 3 2
0

1.0 0.0 1 0
1.0 0.0 1 1
1.0 0.0 1 2
1.0 0.0 1 3
1.0 0.0 1 4
1.0 0.0 2 0
1.0 0.0 2 1
1.0 0.0 2 2
1.0 0.0 2 3
1.0 0.0 2 4
1.0 0.0 3 0
1.0 0.0 3 1
1.0 0.0 3 2
1.0 0.0 3 3
1.0 0.0 3 4
1.0 0.0 4 0
1.0 0.0 4 1
1.0 0.0 4 2
1.0 0.0 4 3
1.0 0.0 4 4
acabou linha
7.0 1.0 0 0
7.0 1.0 0 1
7.0 1.0 0 2
7.0 1.0 0 3
7.0 1.0 0 4
7.0 1.0 1 0
7.0 1.0 1 1
7.0 1.0 1 2
7.0 1.0 1 3
7.0 1.0 1 4
7.0 1.0 2 0
7.0 1.0 2 1
7.0 1.0 2 2
7.0 1.0 2 3
7.0 1.0 2 4
7.0 1.0 3 0
7.0 1.0 3 1
7.0 1.0 3 2
7.0 1.0 3 3
7.0 1.0 3 4
7.0 1.0 4 0
7.0 1.0 4 1
7.0 1.0 4 2
7.0 1.0 4 3
7.0 1.0 4 4
acabou linha
1.0 2.0 0 0
1.0 2.0 0 1
1.0 2.0 0 2
1.0 2.0 0 3
1.0 2.0 0 4
1.0 2.0 1 0
1.0 2.0 1 1
1.0 2.0 1 2
1.0 2.0 1 3
1.0 2.0 1 4
1.0 2.0 2 0
1.0 2.0 2 1
1.0 2.0 2 2
1.0 2.0 2 3
1.0 2.0 2 4
1.0 2.0 3 0
1.0 2.0 3 1
1.0 2.0 3 2
1.0 2.0 3 3
1.0 2.0 3 4
1.0 2.0 4 0
1.0 2.0 4 1
1.0 2.0 4 2
1.0 2.0 4 3
1.0 2.0 4 4
acabou linha
3.0 2.0 0 0
3.0 2.0 0 1
3.0 2.0 0 2
3.0 2.0 0 3
3.0 2.0 0 4
3.0 2.0 1 0
3.0 2.0 1 1
3.0 2.0 1 2
3.0 2.0 1 3
3.0 2.0 1 4
3

In [222]:
results_test = pd.DataFrame(final)

In [223]:
results_test

,home_name,away_name,home_score,away_score,home_pos,away_pos,home_last5all_home,away_last5all_away,last5all_home_away_dif,fifa_home_ova,fifa_home_att,fifa_home_mid,fifa_home_def,fifa_away_ova,fifa_away_att,fifa_away_mid,fifa_away_def,elo_home_score,elo_away_score,tfm_value_home,tfm_value_away,aprov_5_home,aprov_5_away
2140,210.0,94.0,0.0,0.0,10.0,18.0,3.0,-4.0,7.0,67.0,65.0,67.0,68.0,69.0,70.0,68.0,68.0,1430.848145,1437.770264,14130000.0,14900000.0,80.0,26.0
2140,210.0,94.0,0.0,1.0,10.0,18.0,3.0,-4.0,7.0,67.0,65.0,67.0,68.0,69.0,70.0,68.0,68.0,1430.848145,1437.770264,14130000.0,14900000.0,80.0,26.0
2140,210.0,94.0,0.0,2.0,10.0,18.0,3.0,-4.0,7.0,67.0,65.0,67.0,68.0,69.0,70.0,68.0,68.0,1430.848145,1437.770264,14130000.0,14900000.0,80.0,26.0
2140,210.0,94.0,0.0,3.0,10.0,18.0,3.0,-4.0,7.0,67.0,65.0,67.0,68.0,69.0,70.0,68.0,68.0,1430.848145,1437.770264,14130000.0,14900000.0,80.0,26.0
2140,210.0,94.0,0.0,4.0,10.0,18.0,3.0,-4.0,7.0,67.0,65.0,67.0,68.0,69.0,70.0,68.0,68.0,1430.848145,1437.770264,14130000.0,14900000.0,80.0,26.0
2140,210.0,94.0,1.0,0.0,10.0,18.0,3.0,-4.0,7.0,67.0,65.0,67.0,68.0,69.0,70.0,68.0,68.0,1430.848145,1437.770264,14130000.0,14900000.0,80.0,26.0
2140,210.0,94.0,1.0,1.0,10.0,18.0,3.0,-4.0,7.0,67.0,65.0,67.0,68.0,69.0,70.0,68.0,68.0,1430.848145,1437.770264,14130000.0,14900000.0,80.0,26.0
2140,210.0,94.0,1.0,2.0,10.0,18.0,3.0,-4.0,7.0,67.0,65.0,67.0,68.0,69.0,70.0,68.0,68.0,1430.848145,1437.770264,14130000.0,14900000.0,80.0,26.0
2140,210.0,94.0,1.0,3.0,10.0,18.0,3.0,-4.0,7.0,67.0,65.0,67.0,68.0,69.0,70.0,68.0,68.0,1430.848145,1437.770264,14130000.0,14900000.0,80.0,26.0
2140,210.0,94.0,1.0,4.0,10.0,18.0,3.0,-4.0,7.0,67.0,65.0,67.0,68.0,69.0,70.0,68.0,68.0,1430.848145,1437.770264,14130000.0,14900000.0,80.0,26.0


In [224]:
pred = clf.predict_proba(results_test)

In [225]:
results_test['pred_Away'] = pred[:,0]
results_test['pred_Draw'] = pred[:,1]
results_test['pred_Home'] = pred[:,2]

In [229]:
results_test[['home_score', 'away_score', 'pred_Home','pred_Draw','pred_Away']][25:50]

,home_score,away_score,pred_Home,pred_Draw,pred_Away
769,0.0,0.0,0.321431,0.494969,0.183600
769,0.0,1.0,0.208275,0.428136,0.363588
769,0.0,2.0,0.114931,0.203637,0.681431
769,0.0,3.0,0.101731,0.194975,0.703294
769,0.0,4.0,0.101731,0.194975,0.703294
769,1.0,0.0,0.519243,0.360539,0.120217
769,1.0,1.0,0.276088,0.504717,0.219195
769,1.0,2.0,0.150207,0.246087,0.603706
769,1.0,3.0,0.137006,0.237425,0.625569
769,1.0,4.0,0.137006,0.237425,0.625569


In [172]:
clf.classes_

array(['A', 'D', 'H'], dtype=object)